<a href="https://colab.research.google.com/github/benjaminbrown038/Amazon/blob/main/notebooks/amazon/amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sagemaker

- Hugging Face
- PyTorch
- Tensorflow

## Sagemaker

#### Imports

In [ ]:
%%capture
!pip3 install sagemaker
import logging
import time

import boto3

import sagemaker
from sagemaker.xgboost import XGBoost
from sagemaker.experiments.run import Run
from sagemaker.analytics import ExperimentAnalytics

#### Role

In [ ]:
role = sagemaker.get_execution_role()
role

#### Region

In [ ]:
region = boto3.Session().region_name
region

#### Session

In [ ]:
sagemaker_session = sagemaker.Session()
sagemaker_session

#### Bucket

In [ ]:
bucket_name = sagemaker_session.default_bucket()
bucket_name

In [ ]:
prefix = "end-to-end-ml"

## Hugging Face

#### Imports

In [ ]:
!pip3 install sagemaker --upgrade
import sagemaker
from sagemaker.huggingface import HuggingFace, TrainingCompilerConfig
import sagemaker
import boto3
import transformers
import datasets
import argparse
import os
from sagemaker.huggingface import HuggingFace
from sagemaker.s3 import S3Downloader

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='role-name-of-your-iam-role-with-right-permissions')['Role']['Arn']
sess = sagemaker.Session()

In [ ]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--per_device_train_batch_size", type=int, default=32)
    parser.add_argument("--model_name_or_path", type=str)

    # data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--training_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--test_dir", type=str, default=os.environ["SM_CHANNEL_TEST"])

In [ ]:
hyperparameters={'epochs': 1,
                 'per_device_train_batch_size': 32,
                 'model_name_or_path': 'distilbert-base-uncased'}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        hyperparameters = hyperparameters)

In [ ]:
huggingface_estimator.fit(
  {'train': 's3://sagemaker-us-east-1-558105141721/samples/datasets/imdb/train',
   'test': 's3://sagemaker-us-east-1-558105141721/samples/datasets/imdb/test'})

In [ ]:
/opt/conda/bin/python train.py --epochs 1 --model_name_or_path distilbert-base-uncased --per_device_train_batch_size 32

In [ ]:
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data,
    local_path='.',
    sagemaker_session=sess)

In [ ]:
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3dn.24xlarge',
        instance_count=2,
        role=role,
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        hyperparameters = hyperparameters,
        distribution = distribution)

In [ ]:
mpi_options = {
    "enabled" : True,
    "processes_per_host" : 8}

In [ ]:
smp_options = {
    "enabled":True,
    "parameters": {
        "microbatches": 4,
        "placement_strategy": "spread",
        "pipeline": "interleaved",
        "optimize": "speed",
        "partitions": 4,
        "ddp": True,
    }
}

In [ ]:
distribution={
    "smdistributed": {"modelparallel": smp_options},
    "mpi": mpi_options}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3dn.24xlarge',
        instance_count=2,
        role=role,
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        hyperparameters = hyperparameters,
        distribution = distribution
)

In [ ]:
hyperparameters={'epochs': 1,
                 'train_batch_size': 32,
                 'model_name':'distilbert-base-uncased',
                 'output_dir':'/opt/ml/checkpoints'}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',
        instance_count=1,
	      checkpoint_s3_uri=f's3://{sess.default_bucket()}/checkpoints'
        use_spot_instances=True,
        max_wait=3600,
        max_run=1000,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        hyperparameters = hyperparameters)

In [ ]:
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.4.2'}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='run_glue.py',
        source_dir='./examples/pytorch/text-classification',
        git_config=git_config,
        instance_type='ml.p3.2xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        hyperparameters=hyperparameters
)

In [ ]:
metric_definitions = [
    {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
    {"Name": "eval_accuracy", "Regex": "eval_accuracy.*=\D*(.*?)$"},
    {"Name": "eval_loss", "Regex": "eval_loss.*=\D*(.*?)$"},
]

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        metric_definitions=metric_definitions,
        hyperparameters = hyperparameters)

In [ ]:
huggingface_estimator=HuggingFace(
    compiler_config=TrainingCompilerConfig())

## PyTorch

## Tensorflow